In [49]:
import sqlite3
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.subplots as ps
import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash.dependencies as dd
import dash_html_components as html
import pandas as pd

In [32]:
def clean_db(df):
    for i in tqdm(range(len(df.columns))):
        if i == 4:
            for j in range(len(df.iloc[:,i])):
                if not 100000 < float(df.iloc[j,i]) < 150000:
                    df.iloc[j,i] = df.iloc[j-1,i]
        else:
            for j in range(len(df.iloc[:,i])):
                if not 0 < float(df.iloc[j,i]) < 100:
                    df.iloc[j,i] = df.iloc[j-1,i]
    return df

In [33]:
conn = sqlite3.connect('data.db')

In [52]:
sql_q = ('select * from from_nano;')
df = pd.read_sql(sql_q, conn)
df.datatime = pd.to_datetime(df.datatime)
df = df.set_index('datatime')

In [53]:
df = clean_db(df)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 96.64it/s]


In [54]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server
app.config.suppress_callback_exceptions = True

In [79]:
def Temp_trace(df, start_date = datetime.now() - timedelta(days=2), end_date = datetime.now(), color1 = '#9467bd'):
    conn = sqlite3.connect('data.db')
    sql_q = ('select * from from_nano;')
    df = pd.read_sql(sql_q, conn)
    df.datatime = pd.to_datetime(df.datatime)
    df = df.set_index('datatime')
    df = clean_db(df)
    df = df[(df.index>=start_date) & (df.index<=end_date)]
    
    TempChart = ps.make_subplots(specs=[[{"secondary_y": True}]])
    
    TempChart.add_scatter(x=df.index, y=df.air_temp1, mode="lines", name="Air Temperature")
    TempChart.add_scatter(x=df.index, y=df.soil_temp, mode="lines", name="Soil Temperature")
    
    TempChart.update_layout(
        title='Temperature Over Time',
        yaxis_title='Temp (C)',
        # xaxis_title='Date',
        legend=dict(
            orientation="h"
        )
    )
    Charts = [
        dcc.Graph(style={'height': 600, 'width': 1200, 'display': 'inline-block', 'vertical-align': 'middle'},
                  figure=TempChart)]
        
    return Charts

In [86]:
initial_start_date = datetime.now() - timedelta(days=2)
initial_end_date = datetime.now()

TempPlots = dbc.Container([
#     dbc.Row([
#         dbc.Col([
#             html.P('Date Range Slider'),
#             dcc.DatePickerRange(
#                 id='dateRange',
#                 min_date_allowed=df.index.min(),
#                 max_date_allowed=df.index.max(),
#                 start_date=initial_start_date,
#                 display_format='DD/MM/Y',
#                 end_date=initial_end_date)
#         ]),
#     ]),
    dbc.Row([
        dcc.Loading(type='graph', id='TempOutput', 
            children=Temp_trace(
                df,
                initial_start_date,
                initial_end_date
            ),
        style={'width': '100%'}
        )
    ]) 
], className="mt-4")

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 96.60it/s]


In [87]:
navbar_item_width = 9

navbar = dbc.NavbarSimple(
    children=[
        dbc.Row([
            dbc.Col([
                dbc.NavItem(dbc.NavLink("Temperature", href="/temperatures",
                                        style={'width': 19*navbar_item_width, 'display': 'inline-block'}))
            ]),
            dbc.Col([
                dbc.NavItem(dbc.NavLink("Store Level", href="/store_level",
                                        style={'width': 11*navbar_item_width, 'display': 'inline-block'})),
            ]),
            dbc.Col([
                dbc.NavItem(dbc.NavLink("Employee Shortfall", href="/employee_shortfall",
                                        style={'width': 18*navbar_item_width, 'display': 'inline-block'})),
            ]),
            dbc.Col([
                dbc.NavItem(dbc.NavLink("Appendix", href="/Appendix",
                                        style={'width': 8*navbar_item_width, 'display': 'inline-block'}))
            ])
        ])
    ],
    brand="Garden Monitor",
    sticky="top",
    expand='lg',
    light=True
)

In [88]:
app.layout = html.Div([dcc.Location(id='url', refresh=False), navbar, html.Div(id='page-content')])

In [89]:
@app.callback(dd.Output('TempOutput', 'children'), [  
                                                        dd.Input('dateRange', 'start_date'),
                                                        dd.Input('dateRange', 'end_date'),
                                                    ])
def _drawTempPlots(start_date, end_date):
    return Temp_trace(df, start_date=start_date, end_date=end_date)

In [90]:
@app.callback(dd.Output('page-content', 'children'), [dd.Input('url', 'pathname')])
def display_page(pathname):
    if str(pathname).lower() == '/temperatures':
        return TempPlots
    elif str(pathname).lower() == '/':
        return TempPlots
#     elif str(pathname).lower() == '/store_level':
#         return StoreLevel
#     elif str(pathname).lower() == '/employee_shortfall':
#         return EmployeeShortFall
#     elif str(pathname).lower() == '/appendix':
#         return Appendix
    else:
        return fl.abort(404)

In [97]:
if __name__ == "__main__":
    app.run_server(debug=True, port=5001, host='0.0.0.0')

Running on http://0.0.0.0:5001/
Running on http://0.0.0.0:5001/
Running on http://0.0.0.0:5001/
Running on http://0.0.0.0:5001/
Running on http://0.0.0.0:5001/
Debugger PIN: 794-029-470
Debugger PIN: 794-029-470
Debugger PIN: 794-029-470
Debugger PIN: 794-029-470
Debugger PIN: 794-029-470
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

In [98]:
%tb

SystemExit: 1